In [1]:
!pip install keras
!pip install tensorflow
!pip install pandas
!pip install matplotlib seaborn

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Bidirectional, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import tensorflow as tf

In [21]:
data = pd.read_csv("./CSV_Files/glucose_data.csv")
if 'Glucose_time' not in data.columns:
    data["Glucose_time"] = pd.to_datetime(data['reading_time'], unit='ms')
    data['Glucose_time'] = pd.to_datetime(data['Glucose_time'], format='%Y-%m-%d %H:%M:%S')
    data = data.drop(columns=['reading_time'])
    data = data.sort_values(by='Glucose_time')

In [22]:
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data["Glucose_time"] = pd.to_datetime(data["Glucose_time"])

In [23]:
df = data[['Glucose_time', 'reading']]
df.set_index('Glucose_time', inplace=True)
df = df.resample('15min').mean().interpolate(method='linear')
df.head()

,reading
Glucose_time,
2019-08-26 19:00:00,108.109200
2019-08-26 19:15:00,293.000000
2019-08-26 19:30:00,254.317493
2019-08-26 19:45:00,180.000000
2019-08-26 20:00:00,185.919312


In [24]:
df.to_csv('./CSV_Files/glucose_data_resampled_15.csv')

In [25]:
df = pd.read_csv("./CSV_Files/glucose_data_resampled_15.csv")
# Drop all the columns which have unnamed in them
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.head()

,Glucose_time,reading
0,2019-08-26 19:00:00,108.109200
1,2019-08-26 19:15:00,293.000000
2,2019-08-26 19:30:00,254.317493
3,2019-08-26 19:45:00,180.000000
4,2019-08-26 20:00:00,185.919312


In [26]:
# If reading_time is present in the dataset, then convert it to a proper date time format
if 'Glucose_time' not in df.columns:
    df['reading_time'] = pd.to_datetime(df['reading_time'], unit='ms')
    df['reading_time'] = pd.to_datetime(df['reading_time'], format='%Y-%m-%d %H:%M:%S')
    df = df.rename(columns={'reading_time': 'Glucose_time'})
    df = df.sort_values(by='Glucose_time')
    df.head()

In [27]:
# Set the Glucose_time to datetime format and set it as the index
df['Glucose_time'] = pd.to_datetime(df['Glucose_time'])
df.set_index('Glucose_time', inplace=True)
df.head()

,reading
Glucose_time,
2019-08-26 19:00:00,108.109200
2019-08-26 19:15:00,293.000000
2019-08-26 19:30:00,254.317493
2019-08-26 19:45:00,180.000000
2019-08-26 20:00:00,185.919312


In [28]:
checked_df = df.copy()

In [29]:
scaler = MinMaxScaler(feature_range=(0, 1))
df['reading'] = scaler.fit_transform(df[['reading']])
df.head()

,reading
Glucose_time,
2019-08-26 19:00:00,0.108470
2019-08-26 19:15:00,0.821940
2019-08-26 19:30:00,0.672669
2019-08-26 19:45:00,0.385888
2019-08-26 20:00:00,0.408729


In [30]:
df.shape

(500, 1)

In [31]:
def prepare_data(time_series_data, n_features):
    X, y = [], []
    for i in range(len(time_series_data)):
        end_ix = i + n_features
        if end_ix > len(time_series_data)-1:
            break
        seq_x, seq_y = time_series_data[i:end_ix], time_series_data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [32]:
time_series_data = df['reading'].values

In [33]:
n_features_lst = [3, 5, 7, 9, 11]
batch_size_lst = [16, 32, 64]
patience_lst = [25]
best_rmse = float('inf')
best_parameter = {}
best_predictions = None
best_model = None

In [34]:
def lr_schedule(epoch, lr):
    return lr * 0.995

In [35]:
results_sheet = pd.DataFrame()

In [36]:
for n_feat in n_features_lst:
    for batch_size in batch_size_lst:
        for patience in patience_lst:
            print(f"Training model with n_features={n_feat}, dropout_rate=0.2, batch_size={batch_size}, patience={patience}")

            # Prepare data
            train_data_scaled = time_series_data[:-10]
            test_data_scaled = time_series_data[-(n_feat + 10):]

            X_train, y_train = prepare_data(train_data_scaled, n_feat)
            X_test, y_test = prepare_data(test_data_scaled, n_feat)

            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
            X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

            # Define the model
            model = Sequential()
            model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_feat, 1)))
            if n_feat > 3:
                model.add(MaxPooling1D(pool_size=2))
            elif n_feat > 2:
                model.add(MaxPooling1D(pool_size=1))
            model.add(Dropout(0.2))

            model.add(LSTM(units=200, activation='tanh', return_sequences=True))
            model.add(Dropout(0.2))
            model.add(BatchNormalization())

            model.add(Bidirectional(LSTM(units=150, activation='relu', return_sequences=True)))
            model.add(Dropout(0.2))
            model.add(BatchNormalization())

            model.add(Bidirectional(LSTM(units=100, activation='relu', return_sequences=True)))
            model.add(Dropout(0.2))
            model.add(BatchNormalization())

            model.add(LSTM(units=50, activation='relu', return_sequences=False))  # Set return_sequences to False
            model.add(Dropout(0.2))
            model.add(BatchNormalization())

            model.add(Dense(100, activation='relu'))
            model.add(Dense(50, activation='relu'))
            model.add(Dense(1))

            # Compile the model
            optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
            model.compile(optimizer=optimizer, loss='mse')

            early_stop = EarlyStopping(monitor='val_loss', patience=25, verbose=1, restore_best_weights=True)
            lr_scheduler = LearningRateScheduler(lr_schedule)

            # Train the model
            history = model.fit(X_train, y_train, epochs=500, batch_size=batch_size, verbose=1, validation_split=0.2, callbacks=[early_stop, lr_scheduler])

            predictions = []
            curr_sequence = X_test[0].reshape(1, n_feat, 1)

            for i in range(10):
                next_pred = model.predict(curr_sequence)
                predictions.append(next_pred[0, 0])

                # Update the sequence: drop the first value and add the new prediction
                curr_sequence = np.append(curr_sequence[:, 1:, :], next_pred.reshape(1, 1, 1), axis=1)

            predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()
            col_name = f"{n_feat}_{batch_size}_{patience}"
            results_sheet[col_name] = predictions

            if 'actual_values' not in results_sheet.columns:
                results_sheet['actual_values'] = checked_df['reading'].values[-10:]

            rmse = np.sqrt(mean_squared_error(checked_df['reading'].values[-10:], predictions))
            print(f"RMSE: {rmse}")

            if rmse < best_rmse:
                best_rmse = rmse
                best_parameter = {'n_features': n_feat, 'dropout_rate': 0.2, 'batch_size': batch_size, 'patience': patience}
                best_predictions = predictions
                best_model = model

print(f"Best parameters: {best_parameter}")
print(f"Best RMSE: {best_rmse}")

# Display the DataFrame
print(results_sheet)

Training model with n_features=3, dropout_rate=0.2, batch_size=16, patience=25


c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 46s 216ms/step - loss: 0.1786 - val_loss: 0.1051 - learning_rate: 9.9500e-04
Epoch 2/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0662 - val_loss: 0.0936 - learning_rate: 9.9003e-04
Epoch 3/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0525 - val_loss: 0.0776 - learning_rate: 9.8507e-04
Epoch 4/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0390 - val_loss: 0.0673 - learning_rate: 9.8015e-04
Epoch 5/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0359 - val_loss: 0.0650 - learning_rate: 9.7525e-04
Epoch 6/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0239 - val_loss: 0.0550 - learning_rate: 9.7037e-04
Epoch 7/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0286 - val_loss: 0.0507 - learning_rate: 9.6552e-04
Epoch 8/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0210 - val_loss: 0.0486 - learning_rate: 9.6069e-04
Epoch 9/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0281 - val_loss: 0.0485 

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
13/13 ━━━━━━━━━━━━━━━━━━━━ 50s 350ms/step - loss: 0.7343 - val_loss: 0.0897 - learning_rate: 9.9500e-04
Epoch 2/500
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 0.1353 - val_loss: 0.1006 - learning_rate: 9.9003e-04
Epoch 3/500
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 0.0542 - val_loss: 0.0916 - learning_rate: 9.8507e-04
Epoch 4/500
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0555 - val_loss: 0.0918 - learning_rate: 9.8015e-04
Epoch 5/500
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0467 - val_loss: 0.0841 - learning_rate: 9.7525e-04
Epoch 6/500
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0310 - val_loss: 0.0803 - learning_rate: 9.7037e-04
Epoch 7/500
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0328 - val_loss: 0.0744 - learning_rate: 9.6552e-04
Epoch 8/500
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0273 - val_loss: 0.0695 - learning_rate: 9.6069e-04
Epoch 9/500
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0310 - val_loss: 0.0660 

...

Epoch 1/500
24/24 ━━━━━━━━━━━━━━━━━━━━ 48s 197ms/step - loss: 0.2552 - val_loss: 0.0863 - learning_rate: 9.9500e-04
Epoch 2/500
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step - loss: 0.0689 - val_loss: 0.0784 - learning_rate: 9.9003e-04
Epoch 3/500
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0617 - val_loss: 0.0600 - learning_rate: 9.8507e-04
Epoch 4/500
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0415 - val_loss: 0.0492 - learning_rate: 9.8015e-04
Epoch 5/500
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0386 - val_loss: 0.0467 - learning_rate: 9.7525e-04
Epoch 6/500
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 74ms/step - loss: 0.0369 - val_loss: 0.0432 - learning_rate: 9.7037e-04
Epoch 7/500
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - loss: 0.0310 - val_loss: 0.0464 - learning_rate: 9.6552e-04
Epoch 8/500
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - loss: 0.0353 - val_loss: 0.0436 - learning_rate: 9.6069e-04
Epoch 9/500
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - loss: 0.0292 - val_loss: 0.0435 

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 39s 373ms/step - loss: 0.2147 - val_loss: 0.0936 - learning_rate: 9.9500e-04
Epoch 2/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - loss: 0.0930 - val_loss: 0.0734 - learning_rate: 9.9003e-04
Epoch 3/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0854 - val_loss: 0.0614 - learning_rate: 9.8507e-04
Epoch 4/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - loss: 0.0588 - val_loss: 0.0574 - learning_rate: 9.8015e-04
Epoch 5/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 0.0590 - val_loss: 0.0547 - learning_rate: 9.7525e-04
Epoch 6/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - loss: 0.0577 - val_loss: 0.0479 - learning_rate: 9.7037e-04
Epoch 7/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0466 - val_loss: 0.0482 - learning_rate: 9.6552e-04
Epoch 8/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0388 - val_loss: 0.0512 - learning_rate: 9.6069e-04
Epoch 9/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0485 - val_loss: 0.0448

c:\Users\BMVSI-138\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 38s 757ms/step - loss: 0.9241 - val_loss: 0.0873 - learning_rate: 9.9500e-04
Epoch 2/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 241ms/step - loss: 0.1909 - val_loss: 0.0731 - learning_rate: 9.9003e-04
Epoch 3/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - loss: 0.1255 - val_loss: 0.0725 - learning_rate: 9.8507e-04
Epoch 4/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.0751 - val_loss: 0.0680 - learning_rate: 9.8015e-04
Epoch 5/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0786 - val_loss: 0.0599 - learning_rate: 9.7525e-04
Epoch 6/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0635 - val_loss: 0.0556 - learning_rate: 9.7037e-04
Epoch 7/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 0.0375 - val_loss: 0.0532 - learning_rate: 9.6552e-04
Epoch 8/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - loss: 0.0478 - val_loss: 0.0509 - learning_rate: 9.6069e-04
Epoch 9/500
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step - loss: 0.0593 - val_loss: 0.0486 - learning_rate

In [37]:
results_sheet.to_csv('./Resampled_Data/Predictions_1_20_15mins.csv')

In [39]:
df_1 = pd.DataFrame({'actual_values': checked_df['reading'].values[-10:], 'predictions': best_predictions})
df_1

,actual_values,predictions
0,321.250000,327.922028
1,318.750000,306.369446
2,317.000000,304.853180
3,328.071429,301.385437
4,339.142857,274.656097
5,318.000000,266.767578
6,307.000000,238.025513
7,281.000000,225.365051
8,273.000000,222.285324
9,260.000000,195.970978


In [22]:
best_predictions = scaler.inverse_transform(np.array(best_predictions).reshape(-1, 1)).flatten()
y_test = scaler.inverse_transform(y_test[:20].reshape(-1, 1)).flatten()

df_1 = pd.DataFrame({'Actual': y_test, 'Predicted': best_predictions})

In [23]:
df_1

,Actual,Predicted
0,333.444444,88234.820312
1,337.555556,88395.273438
2,341.666667,87417.765625
3,337.666667,87229.320312
4,336.000000,86151.898438
5,330.000000,85571.648438
6,324.000000,84076.031250
7,318.000000,83511.453125
8,312.500000,82019.367188
9,307.000000,81156.414062


# Yaha Tak Run Karna Hai

In [ ]:
df_1

In [ ]:
df_1.to_csv('./Resampled_Data/LSTM_Preds_friday.csv')

## This is a trial model

In [ ]:
# I need the train the model on the train_data and predict the next 16 values and compare it with the test_data
best_pars = {'n_features': 25, 'dropout_rate': 0.2, 'units': 100, 'batch_size': 32, 'patience': 20}
n_feat = best_pars['n_features']
dropout_rate = best_pars['dropout_rate']
units = best_pars['units']
batch_size = best_pars['batch_size']
patience = best_pars['patience']

In [ ]:
train_data_scaled = time_series_data[:-20]
test_data_scaled = time_series_data[-(n_feat + 20):]

In [ ]:
def lr_schedule(epoch, lr):
    return lr * 0.995

In [ ]:
X_train, y_train = prepare_data(train_data_scaled, n_feat)
X_test, y_test = prepare_data(test_data_scaled, n_feat)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the model

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_feat, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Bidirectional(LSTM(units=128, activation='relu', return_sequences=True, kernel_regularizer='l2')))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Bidirectional(LSTM(units=64, activation='relu', return_sequences=False, kernel_regularizer='l2')))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(50, activation='relu'))
model.add(Dense(1))

# Compile the model
optimizer = Adam(learning_rate=0.001, clipnorm=1.0)
model.compile(optimizer=optimizer, loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=patience, verbose=1, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(lr_schedule)

# Train the model
history = model.fit(X_train, y_train, epochs=500, batch_size=batch_size, verbose=1, validation_split=0.2, callbacks=[early_stop, lr_scheduler])

In [ ]:
# Generate predictions
predictions = []
curr_sequence = X_test[0].reshape(1, n_feat, 1)

for i in range(20):
    next_pred = model.predict(curr_sequence)

    # Check the shape of next_pred
    if next_pred.shape != (1, 1):
        print(f"Unexpected shape for next_pred: {next_pred.shape}")
        break

    predictions.append(next_pred[0, 0])

    # Update the sequence: drop the first value and add the new prediction
    curr_sequence = np.append(curr_sequence[:, 1:, :], next_pred.reshape(1, 1, 1), axis=1)

# Inverse transform predictions
if len(predictions) > 0:
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Create a DataFrame with actual and predicted values
    new_df = pd.DataFrame({'Actual': y_test.flatten()[-20:], 'Predicted': predictions.flatten()})
    print(new_df)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test[:20], predictions))
rmse

In [ ]:
new_df

In [ ]:
# View the entire new_df
new_df[['Actual', 'Predicted']].to_csv('./Resampled_Data/LSTM_Predictions.csv')

In [ ]:
for n_features in n_features_lst:
    for units in units_lst:
        for batch_size in batch_size_lst:
            for patience in patience_lst:
                print(f"Training model with n_features={n_features}, dropout_rate=0.2, units={units}, batch_size={batch_size}, patience={patience}")

                # Prepare data
                X, y = prepare_data(time_series_data, n_features)
                X = X.reshape((X.shape[0], X.shape[1], 1))

                # Define LSTM model
                model = Sequential()
                model.add(LSTM(units, activation='relu', input_shape=(n_features, 1)))
                model.add(Dropout(0.2))
                model.add(Dense(1))
                model.compile(optimizer='adam', loss='mse')

                    # Fit model with early stopping
                early_stopping = EarlyStopping(monitor='loss', patience=patience, verbose=1)
                model.fit(X, y, epochs=500, batch_size=batch_size, verbose=1, callbacks=[early_stopping])

                    # Make predictions
                test_input = time_series_data[-n_features:].reshape((1, n_features, 1))
                predictions = []
                for i in range(16):
                    prediction = model.predict(test_input, verbose=0)
                    predictions.append(prediction[0])
                    test_input = np.append(test_input[:, 1:, :], [[prediction[0]]], axis=1)
                # Calculate RMSE
                rmse = np.sqrt(mean_squared_error(time_series_data[-16:], predictions))
                print(f"RMSE: {rmse}")
                # Update best model if RMSE improves
                if rmse < best_rmse:
                    best_rmse = rmse
                    best_parameter = {'n_features': n_features, 'dropout_rate': 0.2, 'units': units,
                                      'batch_size': batch_size, 'patience': patience}
                    best_predictions = predictions
                    best_model = model

print(f"Best RMSE: {best_rmse}")
print("Best Parameters:")
print(best_parameter)

In [ ]:
# Converting both the test_data and the predictions to their original scale
test_data = scaler.inverse_transform(test_data.reshape(-1, 1)).flatten()
best_predictions = scaler.inverse_transform(np.array(best_predictions).reshape(-1, 1)).flatten()

answer_df = pd.DataFrame({'test_data': test_data, 'predictions': best_predictions})

In [ ]:
answer_df

In [ ]:
# Plotting the ACF And PACF Plots

### Actual Code

In [ ]:
for n in n_features_lst:
    for pat in patience_lst:
        X, y = prepare_data(time_series_data, n)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        test_size = 18
        val_size = 24
        train_size = X.shape[0] + (n - 5) - test_size - val_size

        X_train, y_train = X[:train_size], y[:train_size]
        X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
        X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

        # Print the shapes of the train, validation and test sets
        print("Train Shape: ", X_train.shape, y_train.shape)
        print("Validation Shape: ", X_val.shape, y_val.shape)
        print("Test Shape: ", X_test.shape, y_test.shape)

        
        # Building the LSTM Model
        model = Sequential()
        model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n, 1)))
        model.add(LSTM(50, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        # Early stopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=pat, verbose=1)

        # Fitting the model
        model.fit(X_train, y_train, epochs = 300, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

        # Choosing the best model based on the validation loss
        predictions = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, predictions))
        if rmse < best_rmse:
            best_rmse = rmse
            best_parameter['n_features'] = n
            best_parameter['patience'] = pat
            best_predictions = predictions
            best_model = model

print("*" * 50)
print("Best Parameters: ", best_parameter)
print("Best RMSE: ", best_rmse)
print("Prediction :- ", best_predictions)

In [ ]:
validation_predictions_in_original_scale = scaler.inverse_transform(best_predictions)
xlst = validation_predictions_in_original_scale.flatten()
xlst

In [ ]:
def get_previous_3_values_mean(df, index):
    # Get the previous 3 values of the index
    previous_3_values = df.iloc[index-3:index]

    # Inverse transform the values
    previous_3_values = scaler.inverse_transform(previous_3_values)

    # Take the mean of the previous 3 values
    return np.mean(previous_3_values)

mean_240 = get_previous_3_values_mean(df, 240)
mean_240

In [ ]:
# Actual values of the validation set
actual_values = df['reading'].values[train_size:train_size+val_size]
actual_values = actual_values.reshape(-1, 1)
actual_values_in_original_scale = scaler.inverse_transform(actual_values)
actual_values_in_original_scale.flatten()

In [ ]:
# Now I want it to be printed in the form of a dataframe with the predicted values with a shift of 1 and the actual values

final = pd.DataFrame()
# Append the time of time series from values 240 to 264
final['time'] = df.index[train_size:train_size+val_size]

# To get the mean of 240 pass the value of 240 in the get_previous_3_values_mean function
mean_240 = get_previous_3_values_mean(df, 240)

# Remove the last value of the predicted values and store it in a variable
last_value = validation_predictions_in_original_scale[-1]
validation_predictions_in_original_scale = validation_predictions_in_original_scale[:-1]

# Append the precited values with a shift of 1 and the predicted value at 240 being the mean of actual values at 237, 238 and 239. Append the mean first and then the predicted values
final['Shifted_prediction'] = [mean_240] + validation_predictions_in_original_scale.flatten().tolist()
final['unshifted_prediction'] = xlst.flatten().tolist()

# Append the actual values
final['actual'] = actual_values_in_original_scale.flatten()

In [ ]:
final

In [ ]:
# Now we train the model again using the best parameters but now we will train it on the entire dataset and exclude the test data from the dataset

X, y = prepare_data(time_series_data, best_parameter['n_features'])
X = X.reshape((X.shape[0], X.shape[1], 1))

# Splitting the data into train, validation and test sets
# Train data should be of 264 values of the data set and not 240
# Validation data is remaining 24 values of the data set
test_size = 18
train_size = X.shape[0] + (best_parameter['n_features'] - 5) - test_size

X_train, y_train = X[:train_size], y[:train_size]

# Building the LSTM Model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(best_parameter['n_features'], 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fitting the model
model.fit(X_train, y_train, epochs = 300, verbose=0)

# Now we will predict the test data
predictions = model.predict(X[train_size:])
rmse = np.sqrt(mean_squared_error(y[train_size:], predictions))

In [ ]:
# Now converting the predictions to the original scale
predictions_in_original_scale = scaler.inverse_transform(predictions)
plst = predictions_in_original_scale.flatten()
plst

In [ ]:
len(predictions_in_original_scale)

In [ ]:
len(X)

In [ ]:
best_parameter

In [ ]:
# Getting the actual values of the test data
actual_values = df['reading'].values[train_size+best_parameter['n_features']:]
actual_values = actual_values.reshape(-1, 1)
actual_values_in_original_scale = scaler.inverse_transform(actual_values)
actual_values_in_original_scale.flatten()

In [ ]:
predictions_in_original_scale = predictions_in_original_scale.flatten()

# We remove the last value of the predicted values and store it in a variable
last_value = predictions_in_original_scale[-1]
predictions_final = predictions_in_original_scale[:-1]

# Now we add the mean of the last 3 values of the training data as the first value of the predictions_final and then append the predictions_final to the predictions_final
mean_264 = get_previous_3_values_mean(df, 264)
predictions_final = [mean_264] + predictions_final.tolist()

# Now we will create a dataframe with the time series and the actual values and the predicted values
final_test = pd.DataFrame()
final_test['time'] = df.index[train_size+best_parameter['n_features']:]
final_test["unshifted_predcition"] = plst
final_test['predicted'] = predictions_final
final_test['actual'] = actual_values_in_original_scale.flatten()

final_test


In [ ]:
# Now we will again train the data on entire dataset using the best parameters and then predict the future 10 values

X, y = prepare_data(time_series_data, best_parameter['n_features'])
X = X.reshape((X.shape[0], X.shape[1], 1))

# Building the LSTM Model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(best_parameter['n_features'], 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fitting the model
model.fit(X, y, epochs = 300, verbose=0)

# Now we will predict the future 10 values
future_predictions = []
last_values = X[-1]
for i in range(10):
    future_predictions.append(model.predict(last_values.reshape(1, best_parameter['n_features'], 1))[0][0])
    last_values = np.append(last_values[1:], future_predictions[-1])

In [ ]:
future_predictions = np.array(future_predictions)
future_predictions = future_predictions.reshape(-1, 1)
future_predictions_in_original_scale = scaler.inverse_transform(future_predictions)

In [ ]:
future_predictions_in_original_scale.flatten()

In [ ]:
# I have a data frame called checked df which has the reading as a column and reading_time as the index
# Now in this dataframe add 10 columns with their time and readings as predicted by the model

# Get the last time of the checked_df
last_time = checked_df.index[-1]

# Get the time of the future predictions
future_time = pd.date_range(start=last_time, periods=10, freq='5min')[0:]

# Create a dataframe with the future time and the future predictions
future_df = pd.DataFrame()
future_df['time'] = future_time

future_df['reading'] = future_predictions_in_original_scale.flatten()

In [ ]:
future_df

In [ ]:
# Rename the columns of future df to match the checked df
future_df.rename(columns={'time': 'reading_time', 'reading': 'reading'}, inplace=True)
future_df.head()

In [ ]:
future_df.set_index('reading_time', inplace=True)

In [ ]:
checked_df.head()

In [ ]:
final_df = pd.concat([checked_df, future_df], ignore_index=False)

In [ ]:
final_df.tail(20)

In [ ]:
# Now we plot the graph of the final_df along with the checked df.

plt.figure(figsize=(20, 10))
sns.lineplot(data=final_df, x=final_df.index, y='reading', color='blue')
sns.lineplot(data=checked_df, x=checked_df.index, y='reading', color='red')

plt.xlabel('Time')
plt.ylabel('Reading')
plt.title('Predicted Glucose Levels')

plt.legend(['Actual+Predicted', 'Actual'])
plt.show()

In [ ]:
n_features = 10

X, y = prepare_data(time_series_data, n_features)

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

In [ ]:
# Building the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_features, 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=17)

history = model.fit(X_train, y_train, epochs=500, verbose=1, validation_data = (X_val, y_val) ,callbacks=[early_stop])

In [ ]:
plt.plot(history.history['loss'], label="train_loss")
plt.plot(history.history['val_loss'], label="val_loss")
plt.legend()

plt.show()

In [ ]:
# Now
# Reshape the last_sequence to match the input shape of the model
last_sequence = X[-1].reshape((1, n_features, 1))

# Predict the next 10 values
predictions = []
for _ in range(10):
    next_value = model.predict(last_sequence)
    predictions.append(next_value[0][0])
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0][-1] = next_value

# To convert this predictions to the actual glucose values, we need to inverse the scaling
predictions = scaler.inverse_transform([predictions])

print(predictions)

In [ ]:
# I want to implement an LSTM model by taking values of n_features from [3, 5, 10, 15, 20] and then compare the results and also want to choose patience values from [5, 10, 15, 20] and compare the results.

# I will use the above code and modify it to take the values of n_features and patience as input and then return the predictions and the model.

# To do this I will apply a loop on the values of n_features and patience and then store the results in a dictionary

# I will then convert the dictionary to a pandas dataframe and then save it as a csv file

for n_features in [3, 5, 10, 15, 20]:
    for patience in [5, 10, 15, 20]:
        X, y = prepare_data(time_series_data, n_features)
        X = X.reshape((X.shape[0], X.shape[1], 1))

        split = int(0.8 * len(X))
        X_train, X_val = X[:split], X[split:]
        y_train, y_val = y[:split], y[split:]

        model = Sequential()
        model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_features, 1)))
        model.add(LSTM(50, activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')

        early_stop = EarlyStopping(monitor='val_loss', patience=patience)

        history = model.fit(X_train, y_train, epochs=500, verbose=1, validation_data = (X_val, y_val) ,callbacks=[early_stop])

        last_sequence = X[-1].reshape((1, n_features, 1))

        predictions = []
        for _ in range(10):
            next_value = model.predict(last_sequence)
            predictions.append(next_value[0][0])
            last_sequence = np.roll(last_sequence, -1, axis=1)
            last_sequence[0][-1] = next_value

        predictions = scaler.inverse_transform([predictions])
        print(predictions)

        # Save the results in a dictionary
        results = {
            'n_features': n_features,
            'patience': patience,
            'predictions': predictions.flatten()
        }

        # Convert the dictionary to a pandas dataframe
        results_df = pd.DataFrame([results])

        # Save the dataframe as a csv file
        results_df.to_csv(f"results_n_features_{n_features}_patience_{patience}.csv")

In [ ]:
predictions.flatten()
scaler.inverse_transform([predictions.flatten()])

In [ ]:
mean_240 = get_previous_3_values_mean(actual_values, 0)
print(mean_240)

In [ ]:
# CGPT Generated Code
def prepare_data(time_series_data, n_features):
    X, y = [], []
    for i in range(len(time_series_data) - n_features):
        seq_x = time_series_data[i:i + n_features]
        seq_y = time_series_data[i + n_features]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# CGPT Generated Code
def train_and_evaluate(n_features, patience):
    # Preparing the dataset
    X, y = prepare_data(df['reading'].values, n_features)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    # Split data into training and validation sets
    X_train, X_val = X[:-17], X[-17:-9]
    y_train, y_val = y[:-17], y[-17:-9]

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_features, 1)))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Define early stopping
    early_stop = EarlyStopping(monitor='val_loss', patience=patience)

    # Train the model
    history = model.fit(X_train, y_train, epochs=500, verbose=1, validation_data=(X_val, y_val), callbacks=[early_stop])

    # Plot training history
    plt.plot(history.history['loss'], label="train_loss")
    plt.plot(history.history['val_loss'], label="val_loss")
    plt.legend()
    plt.show()

    # Evaluate the model on the last 9 values
    X_test, y_test = X[-9:], y[-9:]
    last_sequence = X_test[-1].reshape((1, n_features, 1))
    predictions = []
    for _ in range(9):
        next_value = model.predict(last_sequence)
        predictions.append(next_value[0][0])
        last_sequence = np.roll(last_sequence, -1, axis=1)
        last_sequence[0][-1] = next_value

    # Inverse transform the predictions
    predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
    actuals = scaler.inverse_transform(y_test.reshape(-1, 1))
    print('Predictions:', predictions)
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(actuals, predictions))

    print(f'n_features: {n_features}, patience: {patience}, RMSE: {rmse}')
    return model, rmse, predictions

In [ ]:
n_features_list = [3, 5, 10, 15, 20]
patience_list = [5, 10, 15, 20]

In [ ]:
best_model = None
best_rmse = float('inf')
best_params = {}
best_predictions = None

In [ ]:
for n_features in n_features_list:
    for patience in patience_list:
        model, rmse, predictions = train_and_evaluate(n_features, patience)
        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model
            best_predictions = predictions
            best_params = {'n_features': n_features, 'patience': patience}

In [ ]:
print(f'Best RMSE: {best_rmse}, Best Params: {best_params}')
print('Best Predictions:', best_predictions)


In [ ]:
p = 's*'
p.endswith('*')

In [ ]:
x = ["s", "sh", "sha", "shai", "shaik", "shaikh"]
x[0][0]